In [1]:
# from translate import Translator
from langdetect import detect, detect_langs
from dateutil import parser
import re
import psycopg2
import pandas as pd

import translators as ts
from os import environ

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_12755/21036820.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
conn = psycopg2.connect(
host = environ['DB_HOST'],
port = environ['DB_PORT'],
user = environ['DB_USER'],
password = environ['DB_PASSWORD'],
database = 'blogs')

In [3]:
query = r"""
    SELECT id, raw_html, language, country FROM articles
    -- WHERE article_type = 'blog'
    WHERE iso3 IS NULL
    LIMIT 500
;"""
body = pd.read_sql_query(query, conn)
body

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_93593/870457995.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  body = pd.read_sql_query(query, conn)


,id,raw_html,language,country
0,13296,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,None
1,13331,"<html dir=""ltr"" class="" js"" data-once=""media-l...",es,None
2,13346,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,None
3,13350,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,None
4,5540,"<html lang=""es"" dir=""ltr"" class="" js""><head>\n...",es,None
...,...,...,...,...
495,5366,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,United States of America
496,5309,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,United States of America
497,5340,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,United States of America
498,5370,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,United States of America


In [4]:
reg = r"<p>(.*?)<\/p>"

body['raw'] = body['raw_html'].apply(lambda x: ' '.join(re.findall(reg, x)))

In [5]:
def translate (row):
    lang = row['language']
    # translator = Translator(from_lang = lang, to_lang = 'en')
    # return translator.translate(row['date'].strip().upper())
    if row['country'] is not None:
        if lang != 'en':
            try:
                return ts.translate_text(row['country'].strip().lower(), from_language = lang, to_language = 'en')
            except:
                return row['country'].strip().lower()
        else:
            return row['country'].strip().lower()
    else:
        return None

In [6]:
body['country_translation'] = body.apply(translate, axis = 1)
body

,id,raw_html,language,country,raw,country_translation
0,13296,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,None,United Nations Development Programme,None
1,13331,"<html dir=""ltr"" class="" js"" data-once=""media-l...",es,None,Foto: Cortesía El Sistema <i>Se trata de un ...,None
2,13346,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,None,This Environmental and Social Management Frame...,None
3,13350,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,None,"In January and February 2017, EECR Cluster par...",None
4,5540,"<html lang=""es"" dir=""ltr"" class="" js""><head>\n...",es,None,"&nbsp; El Intendente de San Antonio de Areco, ...",None
...,...,...,...,...,...,...
495,5366,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,United States of America,Development is an uneven process. Cycles of pr...,united states of america
496,5309,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,United States of America,"On July 17, Dominican Republic presented its ...",united states of america
497,5340,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,United States of America,"<i>Travestis</i>, transsexual and transgen...",united states of america
498,5370,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,United States of America,"On March 11, 2020 the World Health Organizatio...",united states of america


In [7]:
# body[body['country_translation'].isna()]
body[(body['country_translation'] == body['country']) & ~(body['country_translation'].isna())]

,id,raw_html,language,country,raw,country_translation
88,2621,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...",es,France,Бул макалада биз сизди...,France
344,2638,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",es,Spain,UNDP Liberia in partnership with iCampus Liber...,Spain
360,2554,"<html lang=""en"" data-rh=""lang""><head><title>دو...",es,Spain,,Spain


In [8]:
# translator = Translator(from_lang = 'es', to_lang = 'en')
# translator.translate(r'21 DE FEBREERO DE 2022')
# ts.translate_text('2 DE MAYO DE 2023', from_language = 'es', to_language = 'en')

In [71]:
cur = conn.cursor()

for i, r in body[body['datetime'].notnull()].iterrows():
    sql = """
        UPDATE articles
        SET parsed_date = %s
        WHERE id = %s
    ;"""
    cur.execute(sql, (r['datetime'], r['id']))

conn.commit()
cur.close()

In [ ]:
## CLOSE THE CONNECTION TO THE DB
conn.close()